In [1]:
# import lib
from IPython.display import Latex
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torchsummary
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Function
import  visdom
import scipy.sparse as sp
import gurobipy as gp
import itertools
import pdb
import datetime
from random import choice
from gurobipy import GRB
import pickle

# Prepare data

In [2]:
def load_data(data_dir='1_layer_weights/256/'):
    dict_param = {}
    dict_param['w0'] = np.sign(np.load(data_dir+"w0.npy")).T
    dict_param['size_hidden'], dict_param['size_input'] = np.shape(dict_param['w0'])
    dict_param['w1'] = np.sign(np.load(data_dir+"w1.npy")).T    
    dict_param['size_output'],_ = np.shape(dict_param['w1'])

    # MNIST dataset
    test_dataset = torchvision.datasets.MNIST(root='data',
                                              train=False,
                                              transform=transforms.ToTensor())
    return dict_param, test_dataset

In [3]:
def set_param(dict_param, test_dataset, restriction=False,  flip_rate=0.5):
    x, label = choice(test_dataset)
    x = x.reshape(28*28).numpy()*2-1
    x = np.sign(x)
    x[x==0] = 1 
    confidence_vec = dict_param['w1']@np.sign(dict_param['w0']@x)
    label_predicted = np.argmax(confidence_vec)
    if restriction is True:
        flips = 2*np.random.binomial(n=1, p=1-flip_rate, size=dict_param['size_input'])-1
        x_turb = x * flips
        target = np.argmax(dict_param['w1']@np.sign(dict_param['w0']@x_turb))
        dict_param['set_const'] = np.nonzero(flips==1)[0]
        dict_param['set_var'] = np.nonzero(flips != 1)[0]
        print('the number of flips is {}'.format(len(dict_param['set_var'])))
    else:
        dict_param['set_var'] = range(dict_param['size_input'])
        dict_param['set_const'] = range(0)
        select_range = np.nonzero(confidence_vec<np.max(confidence_vec))[0] # avoid equal confidence level
        #target = (label_predicted + np.random.randint(9) + 1) % 10
        if len(select_range) == 0:
            raise Exception('Weird! All confidence levels are same!')
        target = select_range[np.random.randint(len(select_range))]
    print(label, label_predicted, target)
    dict_param['target'] = target
    dict_param['label_original'] = label
    dict_param['label_predicted'] = label_predicted
    dict_param['x'] = x

# Build MIP model

In [16]:
class MinistExtendedFormulation(object):
    def __init__(self, dict_param, is_conv=False, is_integral=False, is_branchPriority=False,
                 obj_type=GRB.MINIMIZE, pixel_change_tol=784, model_attrs={}):
        self.model = gp.Model("robustness_BNN")
        self.param = dict_param
        self.is_conv = is_conv
        self.is_integral= is_integral
        self.is_branchPriority = is_branchPriority
        self.obj_type = obj_type
        self.pixel_change_tol = pixel_change_tol
        self.model_attrs = model_attrs
        
        self.vars = {}
        self.vars_incumbent = dict(Status=False)
        self.confidence_level = 0.
        self.result_only = True # save the result only
    
    def setup(self):
        """set up the corresponding model
        """
        self.model.setParam("TimeLimit", 1800)
        self.model.setParam("PreCrush", 1)
#         if self.is_integral is False:
#             #choose barrier method for continuous models
#             self.model.setParam("Method", 2)
#             #Disable crossover to accelarate the algorithm
#             self.model.setParam("Crossover", 0)
            
        for key,value in self.model_attrs.items():
            self.model.setParam(key, value)
            
        model = self.model
        w0 = self.param['w0']
        w1 = self.param['w1']
        x = self.param['x']
        
        model._param = self.param
        model._num_usercut = 0
        
        set_var= self.param['set_var']
        size_hidden = self.param['size_hidden']
        size_output = self.param['size_output']
        
        label_predicted = dict_param['label_predicted']
        target = self.param['target']
        
        type_var = GRB.BINARY if self.is_integral else GRB.CONTINUOUS
        
        x0 = model.addVars(set_var, vtype=type_var, ub=1., name='x0')
        if self.is_branchPriority is True:
            # branch on lower-lever variables first
            for i in x0:
                x0[i].branchPriority = 1
                
        x1 = model.addVars(range(size_hidden), vtype=type_var, ub=1., name='x1')
        
        y0 = model.addVars(set_var, lb=-1., vtype=GRB.CONTINUOUS, name='y0')
        y1 = model.addVars(range(size_hidden), lb=-1., vtype=GRB.CONTINUOUS, name='y1')
        
        self.vars['y0'] = y0
        self.vars['y1'] = y1
        model._vars = self.vars
        #self.vars['z0'] = z0
        
        model.addConstrs(y0[j] == 2*x0[j] - 1. for j in set_var)
        model.addConstrs(y1[j] == 2*x1[j] - 1. for j in range(size_hidden))
        
        if self.is_conv is True:
            self.add_constrs_conv()
        else:
            self.add_constrs_natural()
            
        if self.obj_type == GRB.MINIMIZE:
            model.addConstrs(gp.quicksum(w1[j, k]*y1[k] for k in range(size_hidden)) + self.confidence_level <= 
                          gp.quicksum(w1[target, k]*y1[k] for k in range(size_hidden)) for j in range(size_output) if j != target)
            model.setObjective(len(set_var)/2-gp.quicksum(x[j]*y0[j]/2 for j in set_var), GRB.MINIMIZE)
        elif self.obj_type == GRB.MAXIMIZE:
            model.addConstr(len(set_var)-gp.quicksum(x[j]*y0[j] for j in set_var) <= 2*self.pixel_change_tol)
            model.setObjective(gp.quicksum(w1[target, k]*y1[k] for k in range(size_hidden)) - 
                               gp.quicksum(w1[label_predicted, k]*y1[k] for k in range(size_hidden)), GRB.MAXIMIZE)
    
    def set_confidence_level(self, confidence_level):
        self.confidence_level = confidence_level
        self.model = gp.Model("robustness_BNN")
        self.vars_incumbent = dict(Status=False)
        
    def add_constrs_natural(self):
        model = self.model
        set_var = self.param['set_var']
        set_const = self.param['set_const']
        size_hidden = self.param['size_hidden'] 
        size_input = self.param['size_input'] 
        w = self.param['w0']
        name_vars = ['y0','y1']
        y0,y1 = (self.vars[name] for name in name_vars)
        z_const = self.param['w0'][:, set_const]
        y_const = z_const @ self.param['x'][set_const]
        model.addConstrs(gp.quicksum(w[i,j]*y0[j] for j in set_var) + y_const[i] + size_input >= size_input * y1[i]
                         for i in range(size_hidden))
        model.addConstrs(gp.quicksum(w[i,j]*y0[j] for j in set_var) + y_const[i] - size_input + 1. <= size_input * y1[i]
                         for i in range(size_hidden))
        
    def add_constrs_conv(self):
        model = self.model
        row = self.param['size_hidden'] 
        w = self.param['w0']
        name_vars = ['y0','y1']
        y0,y1= (self.vars[name] for name in name_vars)
        col = self.param['size_input']
             
        set_var = self.param['set_var']
        set_const = self.param['set_const']
        z_const = self.param['w0'][:, set_const]
        y_const = z_const @ self.param['x'][set_const]   
        z_aux = model.addVars(range(row), set_var, lb=-1.0, vtype=GRB.CONTINUOUS)
        
        model.addConstrs(-(1.+y1[i])<=2*z_aux[i,j] for i in range(row) for j in set_var)
        model.addConstrs(2*z_aux[i,j]<=(1.+y1[i]) for i in range(row) for j in set_var)
        model.addConstrs(-(1-y1[i])<=2*(w[i,j]*y0[j]-z_aux[i,j]) for i in range(row) for j in set_var)
        model.addConstrs(2*(w[i,j]*y0[j]-z_aux[i,j])<=(1-y1[i]) for i in range(row) for j in set_var)
        model.addConstrs(2*z_aux.sum(i, '*') + y_const[i] * (1+y1[i])>=0 for i in range(row))
        model.addConstrs(2*(gp.quicksum(w[i,j]*y0[j] for j in set_var)-z_aux.sum(i,'*'))
                         + (y_const[i]+2) * (1-y1[i])<= 0 for i in range(row))
        
#         z1 = model.addVars(range(row), range(col), lb=-1.0, ub=1., vtype=GRB.CONTINUOUS)
#         z2 = model.addVars(range(row), range(col), lb=-1.0, ub=1., vtype=GRB.CONTINUOUS)
#         model.addConstrs(z1[i,j]<=w[i,j]*y0[j] for i in range(row) for j in range(col))
#         model.addConstrs(z1[i,j]<=y1[i] for i in range(row) for j in range(col))
#         model.addConstrs(z2[i,j]>=w[i,j]*y0[j] for i in range(row) for j in range(col))
#         model.addConstrs(z2[i,j]>=y1[i] for i in range(row) for j in range(col))
#         model.addConstrs(col*(y1[i]-1)<=2*gp.quicksum(z1[i,j] for j in range(col)) for i in range(row))
#         model.addConstrs((col+2)*y1[i]+col-2>=2*gp.quicksum(z2[i,j] for j in range(col)) for i in range(row))
        
    def __repr__(self):
        status = self.vars_incumbent['Status']
        s = ''
        if status is False:
            s = s + 'The optimization problem has not been solved. Please try to solve the problem first.'
        elif status != GRB.OPTIMAL:
            s = s + 'The solver is not able to solve the problem to optimality.'
        else:
            s = s + ('The solver is able to solve the problem to optimality within {}.\n'.format(self.vars_incumbent['time']))
            s = s + ('The original input is:\n{}\n'.format(self.param['x']))
            y0,y1 = (self.vars_incumbent[name] for name in ['y0', 'y1'])
            s= s + ('The targeted input is:\n{}\n'.format(y0))
            s= s + ('The hidden layers are:\n{}\n'.format(y1))
        return s
    
    def __call__(self, callback=None):
        if self.vars_incumbent['Status'] is False:
            self.setup()          
            
        if callback is None:
            self.model.optimize()
        elif self.is_integral is False:
            self.model.setParam("OutputFlag", 0)
            self.model._tol = self.model.Params.FeasibilityTol
            #self.model._tol = 1e-5
            self.model._iter_tol = 10000
            self.model._iter = 0
            #self.model._lazy_status = False
            obj_previous = -1e5
            obj_current = 1e5
            while np.abs(obj_previous-obj_current)>self.model._tol*1e-2:
            #while (self.model._lazy_status is False) and (np.abs(obj_previous-obj_current)<=self.model._tol):
                self.model.optimize()
                obj_previous = obj_current
                obj_current = self.model.objVal
                callback(self.model)
        else:
            self.model._tol = 0
            self.model.optimize(callback)
            #self.model.optimize(conv_cb)
        self.vars_incumbent['Status'] = self.model.getAttr('Status')
        if self.vars_incumbent['Status'] == GRB.OPTIMAL:
            print('The objective is {}'.format(self.model.objVal))
            self.set_optimal_sols()
        
    def set_optimal_sols(self, is_print=True):
        model = self.model
        size_hidden = self.param['size_hidden']
        set_var = self.param['set_var'] 
        names_vars = ['y0','y1']
        y0,y1 = (self.vars[name] for name in names_vars)
        
        self.vars_incumbent['y0'] = np.array(model.getAttr("X", y0).values())
        self.vars_incumbent['y1'] = np.array(model.getAttr("X", y1).values())
        self.vars_incumbent['objVal'] = model.objVal
        self.vars_incumbent['time'] = model.getAttr('Runtime')
    
    def save(self, filename, result_only=True):
        model = self.model
        x = self.param['x']
        target = self.param['target']
        label_original = self.param['label_original']
        label_predicted = self.param['label_predicted']
        names_incumbent = ['y0','y1','objVal','time', 'Status']
        dict_instance = {'confidence_level':self.confidence_level, 'label_original':label_original,
                     'label_predicted':label_predicted, 'target':target, 'time':self.model.getAttr('Runtime')}
        try:
            y0,y1,objVal,time,status = (self.vars_incumbent[name] for name in names_incumbent)
            dict_instance.update({'x':x, 'y0':y0, 'y1':y1, 'time':time, 'Status':status, 'objVal':objVal})
        except:
            dict_instance['Status'] = -1
            
        if self.is_integral is True:
            try:
                dict_instance['gap'] = self.model.getAttr('MIPGap')
                dict_instance['bound'] = self.model.getAttr('ObjBound')
                dict_instance['objVal'] = self.model.getAttr('objVal')
                dict_instance['time'] = self.model.getAttr('Runtime')
                dict_instance['Status'] = self.model.getAttr('Status')
            except:
                print('MIP failed to save!')
                dict_instance = {'Status':-1}

        self.result_only = result_only
        if result_only is False:
            # model
            model.write(filename+'.mps')
            # MIP start
            model.write(filename+'.mst')
            # save solutinos
            model.write(filename+'.json')
            
            # save parameters
        with open(filename, 'wb') as f:
                pickle.dump(dict_instance, f)
    
    def load(self, filename):
        with open(filename, 'rb') as f:
            dict_instance = pickle.load(f)
        self.param['x'] = dict_instance['x']
        self.param['target'] = dict_instance['target']
        if self.result_only is False:
            self.model = gp.read(filename+'.mps')
            size_input = self.param['size_input']
            size_hidden = self.param['size_hidden'] 
            names_incumbent = ['y0','y1','objVal','time', 'Status']
            for name in names_incumbent:
                self.vars_incumbent[name] = dict_instance[name]
            self.model.read(filename+'.mst')
            self.vars['y0'] = dict(zip(range(size_input), ((x for x in self.model.getVars() if x.VarName.find('y0') != -1))))
            self.vars['y1'] = dict(zip(range(size_hidden), ((x for x in self.model.getVars() if x.VarName.find('y1') != -1))))
            self.model.update()

In [5]:
def conv_cb(model, where):
    if where == GRB.Callback.MIPNODE:
        status = model.cbGet(GRB.Callback.MIPNODE_STATUS)
        #if (status == GRB.OPTIMAL):
        #if (status == GRB.OPTIMAL) and (model.cbGet(GRB.Callback.MIPNODE_NODCNT)<= 100):
        # only adding cutting planes at the root node
        if (status == GRB.OPTIMAL) and (model.cbGet(GRB.Callback.MIPNODE_NODCNT)==0):
            #pdb.set_trace()        
            m = model._param['size_hidden'] # number of constraints            
            n = model._param['size_input'] # number of variables
            weights = model._param['w0'] # weights matrix
            name_vars = ['y0','y1']
            y0,y1 = (model._vars[name] for name in name_vars)
            
            #pdb.set_trace()
            y0_incumbent = np.array(model.cbGetNodeRel(y0).select())
            y1_incumbent = np.array(model.cbGetNodeRel(y1).select())
            for i in range(m):
                w = weights[i,:]
                t = y1_incumbent[i]
                if np.sum(np.minimum(w*y0_incumbent, t)) < n/2*(t-1) - model._tol:
                    indices = np.where(w*y0_incumbent<t)[0]
                    # (|J|-n/2)t-n/2 <= \sum_{j\in J} w_j*y_j
                    model.cbCut((len(indices)-n/2)*y1[i] - n/2 <= gp.quicksum(weights[i,j]*y0[j] for j in indices))
                    model._num_usercut = model._num_usercut + 1

                if np.sum(np.maximum(w*y0_incumbent, t)) > (n+2)*(t+1)/2-2+model._tol:
                    indices = np.where(w*y0_incumbent>t)[0]
                    # (|J|+1-n/2)t+n/2-1 >= \sum_{j\in J} w_j*y_j
                    #pdb.set_trace()
                    model.cbCut((len(indices)-n/2+1)*y1[i]+n/2-1 >= gp.quicksum(weights[i,j]*y0[j] for j in indices))
                    model._num_usercut = model._num_usercut + 1

In [6]:
def conv_lazy(model, where):
    if where == GRB.Callback.MIPNODE:
        print('MIPNODE')
#         status = model.cbGet(GRB.Callback.MIPNODE_STATUS)
#         if (status == GRB.OPTIMAL):
        #if (status == GRB.OPTIMAL) and (model.cbGet(GRB.Callback.MIPNODE_NODCNT)<= 100):
        # only adding cutting planes at the root node
        #if (status == GRB.OPTIMAL) and (model.cbGet(GRB.Callback.MIPNODE_NODCNT)==0):
            #pdb.set_trace()        
        m = model._param['size_hidden'] # number of constraints            
        n = model._param['size_input'] # number of variables
        weights = model._param['w0'] # weights matrix
        name_vars = ['y0','y1']
        y0,y1 = (model._vars[name] for name in name_vars)
        var_auxiliary = model._vars['auxiliary']

        #pdb.set_trace()
        y0_incumbent = np.array(model.cbGetNodeRel(y0).select())
        y1_incumbent = np.array(model.cbGetNodeRel(y1).select())
        var_auxiliary_incumbent = np.array(model.cbGetNodeRel(var_auxiliary))
        for i in range(m):
            w = weights[i,:]
            t = y1_incumbent[i]
            if np.sum(np.minimum(w*y0_incumbent, t)) < var_auxiliary_incumbent + n/2*(t-1) - model._tol:
                indices = np.where(w*y0_incumbent<t)[0]
                # (|J|-n/2)t-n/2 <= \sum_{j\in J} w_j*y_j
                model.cbLazy((len(indices)-n/2)*y1[i] - n/2 + var_auxiliary <= gp.quicksum(weights[i,j]*y0[j] for j in indices))
                model._num_usercut = model._num_usercut + 1

            if np.sum(np.maximum(w*y0_incumbent, t)) + var_auxiliary_incumbent > (n+2)*(t+1)/2-2+model._tol:
                indices = np.where(w*y0_incumbent>t)[0]
                # (|J|+1-n/2)t+n/2-1 >= \sum_{j\in J} w_j*y_j
                #pdb.set_trace()
                model.cbLazy((len(indices)-n/2+1)*y1[i]+n/2-1 >= var_auxiliary + gp.quicksum(weights[i,j]*y0[j] for j in indices))
                model._num_usercut = model._num_usercut + 1

In [7]:
def add_cut(model):
    m = model._param['size_hidden'] # number of constraints            
    n = model._param['size_input'] # number of variables
    weights = model._param['w0'] # weights matrix
    name_vars = ['y0','y1']
    y0,y1 = (model._vars[name] for name in name_vars)

    #pdb.set_trace()
    y0_incumbent = np.array(model.getAttr("X", y0).values())
    y1_incumbent = np.array(model.getAttr("X", y1).values())
    model._iter = model._iter + 1
    #model._lazy_status = True
    for i in range(m):
        w = weights[i,:]
        t = y1_incumbent[i]
        if np.sum(np.minimum(w*y0_incumbent, t)) < n/2*(t-1) - model._tol:
            indices = np.where(w*y0_incumbent<t)[0]
            # (|J|-n/2)t-n/2 <= \sum_{j\in J} w_j*y_j
            model.addConstr((len(indices)-n/2)*y1[i] - n/2 <= gp.quicksum(weights[i,j]*y0[j] for j in indices))
            model._num_usercut = model._num_usercut + 1
            #model._lazy_status = False

        if np.sum(np.maximum(w*y0_incumbent, t)) > (n+2)*(t+1)/2-2+model._tol:
            indices = np.where(w*y0_incumbent>t)[0]
            # (|J|+1-n/2)t+n/2-1 >= \sum_{j\in J} w_j*y_j
            #pdb.set_trace()
            model.addConstr((len(indices)-n/2+1)*y1[i]+n/2-1 >= gp.quicksum(weights[i,j]*y0[j] for j in indices))
            model._num_usercut = model._num_usercut + 1
            #model._lazy_status = False

# Experiments with varying changing tolerance

In [ ]:
obj_conv = {}
obj_natural = {}
obj_conv_int = {}
obj_natural_int = {}

# specify your own directory to place the experiment results
root_dir = 'experiment_results/robustness_integral_size_hidden1_max_with_varying_pixel_tol/'
for size_hidden in [ 512,]:
    data_dir = '1_layer_weights/' + str(size_hidden) + '/'
    dict_param, test_dataset = load_data(data_dir=data_dir)
    for idx_instance in range(5):
        set_param(dict_param, test_dataset)
        pixel_change_range = [2,3,4] # allowed to change about 5% of pixels
        print('Hidden size:{}, instance index:{}'.format(size_hidden, idx_instance+1)+'*'*40+'{}'.format(datetime.datetime.now()))
        for pixel_change_tol in pixel_change_range:
            print('Hidden size:{}, instance index:{}, tolerance:{}'.format(size_hidden, idx_instance+1, pixel_change_range)
                  +'*'*40+'{}'.format(datetime.datetime.now()))
            instance_natural = MinistExtendedFormulation(dict_param, is_integral=False,
                                                         obj_type=GRB.MAXIMIZE, pixel_change_tol=pixel_change_tol)
            print('\n NATURAL RELAXATION STARTS TO SOLVE'+'-'*20
                  +'# hidden neurons'+str(size_hidden)+','+'#instance'+str(idx_instance+1)+',tol'+str(pixel_change_tol)+'\n')
            instance_natural()
            try:
                instance_natural.save(root_dir+'nat{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol)))                              
                obj_natural[str(size_hidden), str(idx_instance), str(pixel_change_tol)] = instance_natural.vars_incumbent['objVal']
            except KeyError:
                print('Oops! The model is not optimal!')
                obj_natural[str(size_hidden), str(idx_instance)] = -1           


            instance_conv = MinistExtendedFormulation(dict_param, is_integral=False,
                                                      obj_type=GRB.MAXIMIZE, pixel_change_tol=pixel_change_tol)
            print('\n STRONG RELAXATION STARTS TO SOLVE!'+'-'*20
                  +'# hidden neurons'+str(size_hidden)+','+'#instance'+str(idx_instance+1)+',tol'+str(pixel_change_tol)+'\n')
            instance_conv(add_cut)
            try:
                instance_conv.save(root_dir+'conv{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol)))
                obj_conv[str(size_hidden), str(idx_instance), str(pixel_change_tol)] = instance_conv.vars_incumbent['objVal']
            except KeyError:
                print('Oops! The model is not optimal!')
                obj_conv[str(size_hidden), str(idx_instance)] = -1

            instance_natural_int = MinistExtendedFormulation(dict_param, is_integral=True,
                                                             obj_type=GRB.MAXIMIZE, pixel_change_tol=pixel_change_tol)
            print('\n NATURAL MIP STARTS TO SOLVE!'+'-'*20
                  +'# hidden neurons'+str(size_hidden)+','+'#instance'+str(idx_instance+1)+',tol'+str(pixel_change_tol)+'\n')
            instance_natural_int()
            try:
                instance_natural_int.save(root_dir+'nat_int{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol)))                              
                obj_natural_int[str(size_hidden), str(idx_instance), str(pixel_change_tol)] = instance_natural_int.vars_incumbent['objVal']
            except KeyError:
                print('Oops! The model is not optimal!')
                obj_natural_int[str(size_hidden), str(idx_instance)] = -1

            instance_conv_int = MinistExtendedFormulation(dict_param, is_integral=True,
                                                          obj_type=GRB.MAXIMIZE, pixel_change_tol=pixel_change_tol)
            print('\n STRONG MIP STARTS TO SOLVE!'+'-'*20
                  +'# hidden neurons'+str(size_hidden)+','+'#instance'+str(idx_instance+1)+',tol'+str(pixel_change_tol)+'\n')
            instance_conv_int(conv_cb)
            try:
                instance_conv_int.save(root_dir+'conv_int{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol)))                              
                obj_conv_int[str(size_hidden), str(idx_instance), str(pixel_change_tol)] = instance_conv_int.vars_incumbent['objVal']
            except KeyError:
                print('Oops! The model is not optimal!')
                obj_conv_int[str(size_hidden), str(idx_instance)] = -1

In [ ]:
def format_decimal(x):    
    if x==np.inf:
        s = '-'
    else:
        s = '{:.2f}'.format(x)
    return s
def format_int(x):
    if x==np.inf:
        s = '-'
    else:
        s = '{:d}'.format(int(x))
    return s
experiment_frame = pd.DataFrame({'dim_hidden':[], 'pixel_change_tol':[], 'label':[],
                                 'nat_relax':[], 'nat_obj':[], 'nat_bound':[], 'nat_gap':[], 'nat_time':[],
                                 'conv_relax':[], 'conv_obj':[], 'conv_bound':[], 'conv_gap':[], 'conv_time':[], 'rimp':[]})
root_dir = 'experiment_results/robustness_integral_size_hidden1_max_with_varying_pixel_tol/'
n=0
for size_hidden in [32,64,128,256]:
    dim_hidden = size_hidden
    pixel_change_range = [1,2,3,4,5]
    for pixel_change_tol in pixel_change_range:
        for idx_instance in range(5):
            filename = root_dir + 'nat{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol))
            with open(filename, 'rb') as f:
                d = pickle.load(f)
            label = (d['label_original'], d['label_predicted'], d['target'])
            nat_relax = d['objVal']

            filename = root_dir + 'conv{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol))
            with open(filename, 'rb') as f:
                d = pickle.load(f)
                
            try:
                conv_relax = d['objVal']
            except:
                conv_relax = np.nan
#             if d['Status'] != -1:
#                 conv_relax = d['objVal']
#             else:
#                 conv_relax = np.nan

            filename = root_dir + 'nat_int{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol))
            with open(filename, 'rb') as f:
                d = pickle.load(f)
            nat_obj = d['objVal']
            nat_bound = d['bound']
            nat_gap = d['gap']*100
            nat_time = d['time']

            filename = root_dir + 'conv_int{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol))
            with open(filename, 'rb') as f:
                d = pickle.load(f)
            conv_obj = d['objVal']
            conv_bound = d['bound']
            conv_gap = d['gap']*100
            conv_time = d['time']
            rimp = 100*(nat_relax-conv_relax)/(nat_relax-nat_obj + (np.abs(nat_relax-nat_obj)<=1e-4))

            list_frame = ['','', label,  nat_relax,  nat_obj, nat_bound, nat_gap, nat_time,
                                       conv_relax, conv_obj, conv_bound, conv_gap, conv_time, rimp]
            if idx_instance == 0:
                list_frame[0] = dim_hidden
                list_frame[1] = pixel_change_tol

            experiment_frame.loc[n] = list_frame
            n= n + 1
        list_frame = ['', '', 'Ave'] + list(experiment_frame[-5:].mean())
        experiment_frame.loc[n] = list_frame
        n = n + 1
    
        
formatters = [None,  None, None,format_decimal, format_int, format_int, format_int, format_decimal, format_decimal,
              format_int, format_int, format_int, format_decimal, format_decimal]        
latex_str = experiment_frame.to_latex(index=False, formatters=formatters)
print(latex_str)

In [23]:
experiment_frame.to_csv(root_dir+'results.csv', index=False)